# Deploy comunication compression scheme in FedLab

This tutorial provides comprehensive examples about implementing a communication efficiency scheme in FedLab. 

We take the baseline gradient compression algorithms as examples (top-k for gradient sparsification and QSGD for gradient quantization).

## Compress example

In [1]:
import sys
sys.path.append("../")

from fedlab.contrib.compressor.quantization import QSGDCompressor
from fedlab.contrib.compressor.topk import TopkCompressor
import torch

tpk_compressor = TopkCompressor(compress_ratio=0.05) # top 5% gradient
qsgd_compressor = QSGDCompressor(n_bit=8)


ModuleNotFoundError: No module named 'torch'

In [ ]:
# top-k
tensor = torch.randn(size=(100,))
shape = tensor.shape
print("To be compressed tensor:", tensor)

# compress
values, indices = tpk_compressor.compress(tensor)
print("Compressed results top-k values:",values)
print("Compressed results top-k indices:", indices)

# decompress
decompressed = tpk_compressor.decompress(values, indices, shape)
print("Decompressed results:", decompressed)

To be compressed tensor: tensor([ 7.7551e-01, -2.5789e+00, -1.2139e+00,  6.6635e-01,  1.8937e-01,
         1.0737e-03,  5.1149e-01,  3.0985e-01,  1.5327e-01, -1.3022e+00,
         3.6320e-01, -8.1719e-01, -1.7236e-01,  3.6765e-01,  8.8025e-01,
         5.8854e-01,  9.2844e-01,  4.0012e-02, -9.9287e-01,  6.1850e-02,
         2.7857e+00,  1.2063e+00,  1.2431e+00,  1.4535e-01, -8.4183e-01,
        -3.4871e-02, -3.2011e-02, -6.3319e-01, -3.2209e-01,  2.2956e+00,
         2.6015e-01, -1.9257e+00, -6.8645e-01,  1.2412e+00,  2.6703e-02,
        -1.6042e+00, -3.5102e-01, -1.4851e+00, -1.5193e+00, -5.9978e-01,
        -1.4323e+00,  2.0093e-01,  4.9522e-01,  1.0142e+00, -1.3590e+00,
        -1.1961e+00,  8.6942e-01, -3.2653e-02, -1.0595e+00, -1.1150e+00,
         6.6425e-01,  2.9663e-01,  8.4232e-01,  1.1350e-02,  1.0860e+00,
         6.4058e-01, -1.8474e+00, -9.2868e-02,  2.0592e+00,  1.1361e+00,
        -9.5078e-03,  1.2869e-01,  3.9442e-01, -6.9267e-01, -6.6260e-01,
        -5.5249e-01, -5.01

In [ ]:
# qsgd
tensor = torch.randn(size=(100,))
shape = tensor.shape
print("To be compressed tensor:", tensor)

# compress
norm, signs, values = qsgd_compressor.compress(tensor)
print("Compressed results QSGD norm:", norm)
print("Compressed results QSGD signs:", signs)
print("Compressed results QSGD values:", values)


To be compressed tensor: tensor([ 0.6994, -1.1251, -0.9299, -0.6352,  0.2741,  1.5824,  0.0894,  1.5732,
         0.7831, -0.0198,  0.2224, -0.4793, -0.2897,  1.6225,  0.7523, -1.7849,
         0.4316, -0.1608, -0.6759,  1.0389,  0.1073,  1.1752,  0.1218, -0.3599,
        -1.3481, -0.4494, -0.5059,  1.2226,  0.7338, -0.2230,  0.6289,  0.5322,
        -0.1944,  0.7647,  0.5396,  0.5470, -0.1616,  1.4421, -0.2432,  1.8598,
        -1.2581, -0.7223, -0.5471,  1.6624,  0.7318, -0.0626,  1.6238, -0.8099,
        -0.1461,  0.4893,  0.9783, -0.0033, -1.4855, -0.2551,  0.1569, -0.3270,
         0.0954,  1.3373, -1.3616, -0.6106, -0.3886,  0.0768,  0.0695, -1.7971,
        -1.2371,  0.7987,  0.8369,  1.0450, -0.6797, -0.2286,  1.4056, -1.3860,
         0.2538,  0.3095, -1.7382,  1.2275,  0.4352,  0.1406, -0.8640,  0.1920,
         1.4031,  0.2131, -0.0194, -1.2062,  0.8262, -0.3393,  2.3612,  1.2369,
         0.4217, -0.7969,  0.2939, -0.8274, -1.1822,  0.0056, -1.5248,  0.9979,
         0.7542

In [ ]:
# decompress
decompressed = qsgd_compressor.decompress([norm, signs, values])
print("Decompressed results:", decompressed)

Decompressed results: tensor([ 0.7010, -1.1253, -0.9316, -0.6364,  0.2767,  1.5864,  0.0922,  1.5772,
         0.7840, -0.0277,  0.2214, -0.4796, -0.2859,  1.6233,  0.7563, -1.7801,
         0.4335, -0.1660, -0.6733,  1.0423,  0.1107,  1.1806,  0.1291, -0.3597,
        -1.3466, -0.4520, -0.5073,  1.2267,  0.7379, -0.2306,  0.6272,  0.5350,
        -0.1937,  0.7563,  0.5350,  0.5442, -0.1568,  1.4389, -0.2398,  1.8539,
        -1.2636, -0.7287, -0.5442,  1.6602,  0.7379, -0.0553,  1.6233, -0.8024,
        -0.1476,  0.4888,  0.9777, -0.0092, -1.4850, -0.2490,  0.1568, -0.3228,
         0.0922,  1.3374, -1.3559, -0.6088, -0.3966,  0.0830,  0.0646, -1.7986,
        -1.2359,  0.8024,  0.8393,  1.0423, -0.6733, -0.2306,  1.4112, -1.3927,
         0.2583,  0.3136, -1.7432,  1.2267,  0.4335,  0.1476, -0.8578,  0.1937,
         1.4020,  0.2121, -0.0184, -1.2083,  0.8301, -0.3413,  2.3612,  1.2359,
         0.4243, -0.7932,  0.2859, -0.8301, -1.1806,  0.0000, -1.5311,  0.9961,
         0.7563, -

## Use compressor in federated learning

For example on the client side, we could compress the tensors are to compressed and upload the compressed results to server. And server could decompress the tensors follows the compression agreements.

In jupyter notebook, we take the standalone scenario as example.

In [ ]:
from fedlab.contrib.algorithm.basic_client import SGDSerialClientTrainer, SGDClientTrainer
from fedlab.contrib.algorithm.basic_server import SyncServerHandler

class CompressSerialClientTrainer(SGDSerialClientTrainer):
    def setup_compressor(self, compressor):
        #self.compressor = TopkCompressor(compress_ratio=k)
        self.compressor = compressor

    @property
    def uplink_package(self):
        package = super().uplink_package
        new_package = []
        for content in package:
            pack = [self.compressor.compress(content[0])]
            new_package.append(pack)
        return new_package

class CompressServerHandeler(SyncServerHandler):
    def setup_compressor(self, compressor, type):
        #self.compressor = TopkCompressor(compress_ratio=k)
        self.compressor = compressor
        self.type = type

    def load(self, payload) -> bool:
        if self.type == "topk":
            values, indices = payload[0]
            decompressed_payload = self.compressor.decompress(values, indices, self.model_parameters.shape)

        if self.type == "qsgd":
            n, s, l = payload[0]
            decompressed_payload = self.compressor.decompress((n,s,l))
        
        return super().load([decompressed_payload])

In [ ]:
# main, this part we follow the pipeline in pipeline_tutorial.ipynb
# But replace the hander and trainer by the above defined for communication compression

# configuration
from opcode import cmp_op
from munch import Munch
from fedlab.models.mlp import MLP

model = MLP(784, 10)
args = Munch

args.total_client = 100
args.alpha = 0.5
args.seed = 42
args.preprocess = False
args.cuda = False
args.cmp_op = "qsgd" # "topk, qsgd"

args.k = 0.1 # topk
args.bit = 8 # qsgd

if args.cmp_op == "topk":
    compressor = TopkCompressor(args.k)

if args.cmp_op == "qsgd":
    compressor = QSGDCompressor(args.bit)

from torchvision import transforms
from fedlab.contrib.dataset.partitioned_mnist import PartitionedMNIST

fed_mnist = PartitionedMNIST(root="../datasets/mnist/",
                             path="../datasets/mnist/fedmnist/",
                             num_clients=args.total_client,
                             partition="noniid-labeldir",
                             dir_alpha=args.alpha,
                             seed=args.seed,
                             preprocess=args.preprocess,
                             download=True,
                             verbose=True,
                             transform=transforms.Compose([
                                 transforms.ToPILImage(),
                                 transforms.ToTensor()
                             ]))

dataset = fed_mnist.get_dataset(0)  # get the 0-th client's dataset
dataloader = fed_mnist.get_dataloader(
    0,
    batch_size=128)  # get the 0-th client's dataset loader with batch size 128


In [ ]:
# client
from fedlab.contrib.algorithm.basic_client import SGDSerialClientTrainer, SGDClientTrainer

# local train configuration
args.epochs = 5
args.batch_size = 128
args.lr = 0.1

trainer = CompressSerialClientTrainer(model, args.total_client,
                                 cuda=args.cuda)  # serial trainer
# trainer = SGDClientTrainer(model, cuda=True) # single trainer

trainer.setup_dataset(fed_mnist)
trainer.setup_optim(args.epochs, args.batch_size, args.lr)
trainer.setup_compressor(compressor)

# server
from fedlab.contrib.algorithm.basic_server import SyncServerHandler

# global configuration
args.com_round = 10
args.sample_ratio = 0.1

handler = CompressServerHandeler(model=model,
                            global_round=args.com_round,
                            num_clients=args.total_client,
                            sample_ratio=args.sample_ratio,
                            cuda=args.cuda)
handler.setup_compressor(compressor, args.cmp_op)

TypeError: SyncServerHandler.__init__() got an unexpected keyword argument 'num_clients'

In [ ]:
from fedlab.utils.functional import evaluate
from fedlab.core.standalone import StandalonePipeline

from torch import nn
from torch.utils.data import DataLoader
import torchvision

class EvalPipeline(StandalonePipeline):
    def __init__(self, handler, trainer, test_loader):
        super().__init__(handler, trainer)
        self.test_loader = test_loader

    def main(self):
        while self.handler.if_stop is False:
            # server side
            sampled_clients = self.handler.sample_clients()
            broadcast = self.handler.downlink_package

            # client side
            self.trainer.local_process(broadcast, sampled_clients)
            uploads = self.trainer.uplink_package

            # server side
            for pack in uploads:
                self.handler.load(pack)

            loss, acc = evaluate(self.handler.model, nn.CrossEntropyLoss(),
                                 self.test_loader)
            print("loss {:.4f}, test accuracy {:.4f}".format(loss, acc))


test_data = torchvision.datasets.MNIST(root="../datasets/mnist/",
                                       train=False,
                                       transform=transforms.ToTensor())
test_loader = DataLoader(test_data, batch_size=1024)

standalone_eval = EvalPipeline(handler=handler,
                               trainer=trainer,
                               test_loader=test_loader)
standalone_eval.main()

loss 21.7808, test accuracy 0.2125
loss 17.5083, test accuracy 0.4434
loss 13.4432, test accuracy 0.5187
loss 9.1679, test accuracy 0.6912
loss 7.1598, test accuracy 0.7694
loss 6.1386, test accuracy 0.8406
loss 5.2171, test accuracy 0.8657
loss 5.6939, test accuracy 0.8258
loss 4.9492, test accuracy 0.8340
loss 4.9229, test accuracy 0.8489
